In [1]:
import os
import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

import sys
sys.path.append("..") 
from gcforest.gcforest import GCForest

/Users/alex/anaconda/envs/gc/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


# 数据及参数

In [2]:
random_seed = 42
cv=5
score = 'f1_weighted'

In [3]:
def get_toy_config():
    config = {}
    ca_config = {}
    ca_config["random_state"] = random_seed
    ca_config["max_layers"] = 10
    ca_config["early_stopping_rounds"] = 3
    ca_config["n_classes"] = 6
    ca_config["estimators"] = []
    ca_config["estimators"].append({"n_folds": 5, "type": "RandomForestClassifier", "random_state" : random_seed})
    ca_config["estimators"].append({"n_folds": 5, "type": "RandomForestClassifier", "random_state" : random_seed})
    ca_config["estimators"].append({"n_folds": 5, "type": "RandomForestClassifier", "random_state" : random_seed})
    ca_config["estimators"].append({"n_folds": 5, "type": "RandomForestClassifier", "random_state" : random_seed})
    ca_config["estimators"].append({"n_folds": 5, "type": "DecisionTreeClassifier"})
    ca_config["estimators"].append({"n_folds": 5, "type": "DecisionTreeClassifier"})
    ca_config["estimators"].append({"n_folds": 5, "type": "DecisionTreeClassifier"})
    ca_config["estimators"].append({"n_folds": 5, "type": "LogisticRegression"})
    config["cascade"] = ca_config
    return config

In [4]:
path = os.getcwd()+'/../data/20122018freshwater_four_feature.csv'
data = pd.read_csv(path, na_values = np.nan)

In [5]:
X = data.drop(['本周水质'], axis=1).values # Series
y = data['本周水质'].values.reshape(-1,1) - 1

# 1. 中位数填充缺失值，2.Z-score标准化
clean_pipeline = Pipeline([('imputer', preprocessing.Imputer(missing_values='NaN',strategy="median")),
                           ('std_scaler', preprocessing.StandardScaler())])
X = clean_pipeline.fit_transform(X)

In [6]:
X.shape

(33612, 4)

# k折交叉验证

In [7]:
# function：使用5折交叉验证统计各类别5次平均后的Acc，5次平均后的F1，和模型的总Acc以及总Weighted F1

# Input: 
#     X: 总样本
#     y: 总样本
#     model: function
#     cv: cross_validation的次数
# Output:
#     Acc_mean, 各类别的Acc
#     F1_mean, 各类别F1
#     Support_mean, 各类别预测样本占总样本的比重
#     Acc, 总Acc
#     F1_weighted 总Weighted F1


def kftrain(X, y, model, cv):
    model_name = model.__class__.__name__

    n_samples = X.shape[0]
    n_features = X.shape[1]
    n_class = np.unique(y).shape[0]
    Acc_matrix = np.zeros((n_class, cv)) # 矩阵(6,5) 第i行：类别i的在cv组test data的cv个acc
    F1_matrix = np.zeros((n_class, cv)) # 矩阵(6,5) 第i行：类别i的在cv组test data的cv个acc
    cv_F1_weighted = np.zeros(cv) # 5次的F1_weighted
    cv_Acc = np.zeros(cv) # 5次的Acc
    # 各类别的Support，每个类别的support由5次fold后得到的5个support求平均得到，这里support：预测为该类别的样本占总样本的比例
    # 各类别的F1，由5次fold后得到的5个F1求平均得到
    # 所以，Weighted F1 等于“5次fold得到的5个 Weighted F1 求平均得到” ——等价于—— “6个support*6个F1”得到
    Support_matrix = np.zeros((n_class, cv)) # 矩阵(6,5) 5次fold后各类别的support
    
    # 各类别的Mean，SD
    Acc_mean_class = np.zeros(6)
    Acc_SD_class = np.zeros(6)
    F1_mean_class = np.zeros(6)
    F1_SD_class = np.zeros(6)
    
    k = 0
    skf = StratifiedKFold(n_splits=cv) # 定义5折分层划分器
    
    for train_index, test_index in skf.split(X, y):
        K_train_x, K_test_x = X[train_index], X[test_index]
        K_train_y, K_test_y = y[train_index], y[test_index]
        
        if model_name == 'GCForest':
            model.fit_transform(K_train_x, K_train_y.reshape(K_train_y.shape[0]))
        else:
            model.fit(K_train_x, K_train_y)
    
        K_test_y_pred = model.predict(K_test_x)
        
        # 由混淆矩阵计算各类别的Acc
        test_cm = confusion_matrix(K_test_y, K_test_y_pred)
        test_acc_all_class = np.zeros(n_class) # 6个类别上的测试acc
        i = 0
        for c in test_cm:
            test_acc_all_class[i] = c[i]/np.sum(c)
            i += 1
        Acc_matrix[:,k] = test_acc_all_class
        
        # 由classification_report提取f1
        cr = classification_report(K_test_y, K_test_y_pred, digits=4)
        test_f1_all_class = np.zeros(n_class) # 6个类别上的测试f1
        support_all_class = np.zeros(n_class) # 6个类别的support
        i = 0
        for l in range(2,8):
            test_f1_all_class[i] = float(cr.splitlines()[l].split()[3])
            support_all_class[i] = float(cr.splitlines()[l].split()[4])/n_samples
            i = i + 1
        F1_matrix[:,k] = test_f1_all_class
        Support_matrix[:,k] = support_all_class
        
        # cv_F1_weighted[k] = float(cr.splitlines()[9].split()[5]) # 第k折时的Weighted F1
        cv_F1_weighted[k] = f1_score(K_test_y, K_test_y_pred, average="weighted")
        cv_Acc[k] = accuracy_score(K_test_y, K_test_y_pred)
        
        k += 1
    
    Acc_mean = np.mean(Acc_matrix, axis=1) # 6个类别的k-fold平均acc
    
    F1_mean = np.mean(F1_matrix, axis=1) # 6个类别的k-fold平均F1
    Support_mean = np.mean(Support_matrix, axis=1) # 6个类别的k-fold平均support
    
    Acc = np.mean(Acc_mean)
    F1_weighted = np.mean(cv_F1_weighted) # 5个Weighted F1 求平均，并可以证明其等价于 np.sum(F1_mean*Support_mean)
    SD_Acc = np.std(cv_Acc)
    SD_F1 = np.std(cv_F1_weighted)
    
    Acc_mean_class = np.mean(Acc_matrix, axis=1)
    Acc_SD_class = np.std(Acc_matrix, axis=1)
    F1_mean_class = np.mean(F1_matrix, axis=1)
    F1_SD_class = np.std(F1_matrix, axis=1)
    
    return Acc_mean, F1_mean, Support_mean, Acc, F1_weighted, SD_Acc, SD_F1, Acc_mean_class, Acc_SD_class, F1_mean_class, F1_SD_class

In [8]:
config = get_toy_config()

models = [
    LogisticRegression(),
    LinearDiscriminantAnalysis(),
    SVC(probability=True),
    DecisionTreeClassifier(),
    ExtraTreeClassifier(),
    GaussianNB(),
    KNeighborsClassifier(),
    RandomForestClassifier(random_state=random_seed),
    ExtraTreesClassifier(random_state=random_seed),
    GCForest(config)
]

for model in models:
    model_name = model.__class__.__name__
    print(model_name)
    Acc_class, F1_class, Support_class, Acc, F1_weighted, SD_Acc, SD_F1, Acc_mean_class, Acc_SD_class, F1_mean_class, F1_SD_class = kftrain(X, y, model, 5)
    print("===Accuracy===")
    for i in range(len(Acc_class)):
        print("Class %d: %.2f%%" %(i, Acc_class[i]*100))
        print("Mean±SD: %.2f±%.2f%%" %(Acc_mean_class[i]*100,Acc_SD_class[i]*100))
        
    print("Overall: %.2f%%" %(Acc*100))
    print("Mean±SD: %.2f±%.2f%%" %(Acc*100,SD_Acc*100))

    print("===   F1  ===")
    for i in range(len(F1_class)):
        print("Class %d: %.2f%%" %(i, F1_class[i]*100))
        print("Mean±SD: %.2f±%.2f%%" %(F1_mean_class[i]*100,F1_SD_class[i]*100))
        
    print("F{beta}: %.2f%%" %(F1_weighted*100))
    print("Mean±SD: %.2f±%.2f%%" %(F1_weighted*100,SD_F1*100))
    print("\n===============\n")

LogisticRegression


/Users/alex/anaconda/envs/gc/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/alex/anaconda/envs/gc/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/alex/anaconda/envs/gc/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


===Accuracy===
Class 0: 5.74%
Mean±SD: 5.74±3.20%
Class 1: 99.17%
Mean±SD: 99.17±0.33%
Class 2: 57.64%
Mean±SD: 57.64±2.91%
Class 3: 44.13%
Mean±SD: 44.13±3.79%
Class 4: 0.00%
Mean±SD: 0.00±0.00%
Class 5: 75.87%
Mean±SD: 75.87±7.48%
Overall: 47.09%
Mean±SD: 47.09±0.94%
===   F1  ===
Class 0: 10.60%
Mean±SD: 10.60±5.74%
Class 1: 80.26%
Mean±SD: 80.26±0.50%
Class 2: 60.59%
Mean±SD: 60.59±1.72%
Class 3: 51.43%
Mean±SD: 51.43±3.03%
Class 4: 0.00%
Mean±SD: 0.00±0.00%
Class 5: 76.78%
Mean±SD: 76.78±5.36%
F{beta}: 61.59%
Mean±SD: 61.59±0.78%


LinearDiscriminantAnalysis
===Accuracy===
Class 0: 0.00%
Mean±SD: 0.00±0.00%
Class 1: 99.74%
Mean±SD: 99.74±0.25%
Class 2: 40.24%
Mean±SD: 40.24±6.86%
Class 3: 50.88%
Mean±SD: 50.88±3.60%
Class 4: 27.98%
Mean±SD: 27.98±2.00%
Class 5: 44.88%
Mean±SD: 44.88±13.81%
Overall: 43.95%
Mean±SD: 43.95±1.82%
===   F1  ===
Class 0: 0.00%
Mean±SD: 0.00±0.00%
Class 1: 75.44%
Mean±SD: 75.44±1.23%
Class 2: 47.81%
Mean±SD: 47.81±5.96%
Class 3: 58.63%
Mean±SD: 58.63±2.5

/Users/alex/anaconda/envs/gc/lib/python3.6/site-packages/ipykernel/__main__.py:47: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


===Accuracy===
Class 0: 76.70%
Mean±SD: 76.70±4.29%
Class 1: 93.59%
Mean±SD: 93.59±0.77%
Class 2: 89.85%
Mean±SD: 89.85±1.09%
Class 3: 89.77%
Mean±SD: 89.77±1.76%
Class 4: 78.95%
Mean±SD: 78.95±2.81%
Class 5: 88.63%
Mean±SD: 88.63±2.22%
Overall: 86.25%
Mean±SD: 86.25±0.74%
===   F1  ===
Class 0: 77.11%
Mean±SD: 77.11±2.97%
Class 1: 92.00%
Mean±SD: 92.00±0.70%
Class 2: 90.32%
Mean±SD: 90.32±0.70%
Class 3: 90.86%
Mean±SD: 90.86±1.01%
Class 4: 79.92%
Mean±SD: 79.92±2.21%
Class 5: 92.36%
Mean±SD: 92.36±1.42%
F{beta}: 89.78%
Mean±SD: 89.78±0.76%


RandomForestClassifier


...

[ 2019-04-13 15:27:16,721][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_2 - 5_folds.train_4.predict)=99.41%
[ 2019-04-13 15:27:16,728][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_2 - 5_folds.train_cv.predict)=99.36%
[ 2019-04-13 15:27:17,000][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_3 - 5_folds.train_0.predict)=99.26%
[ 2019-04-13 15:27:17,262][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_3 - 5_folds.train_1.predict)=99.42%
[ 2019-04-13 15:27:17,508][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_3 - 5_folds.train_2.predict)=99.37%
[ 2019-04-13 15:27:17,775][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_3 - 5_folds.train_3.predict)=99.52%
[ 2019-04-13 15:27:18,019][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_3 - 5_folds.train_4.predict)=99.31%
[ 2019-04-13 15:27:18,026][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_3 - 5_folds

[ 2019-04-13 15:27:33,407][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_5 - 5_folds.train_0.predict)=98.81%
[ 2019-04-13 15:27:33,679][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_5 - 5_folds.train_1.predict)=98.64%
[ 2019-04-13 15:27:33,921][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_5 - 5_folds.train_2.predict)=98.75%
[ 2019-04-13 15:27:34,198][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_5 - 5_folds.train_3.predict)=98.53%
[ 2019-04-13 15:27:34,440][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_5 - 5_folds.train_4.predict)=98.66%
[ 2019-04-13 15:27:34,447][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_5 - 5_folds.train_cv.predict)=98.68%
[ 2019-04-13 15:27:34,746][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_6 - 5_folds.train_0.predict)=98.79%
[ 2019-04-13 15:27:35,020][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_6 - 5_folds

[ 2019-04-13 15:27:50,960][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_3 - estimator_7 - 5_folds.train_2.predict)=99.44%
[ 2019-04-13 15:27:51,694][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_3 - estimator_7 - 5_folds.train_3.predict)=99.52%
[ 2019-04-13 15:27:52,419][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_3 - estimator_7 - 5_folds.train_4.predict)=99.29%
[ 2019-04-13 15:27:52,425][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_3 - estimator_7 - 5_folds.train_cv.predict)=99.39%
[ 2019-04-13 15:27:52,434][cascade_classifier.calc_f1] Weighted F1 (layer_3 - train.classifier_average)=99.37%
[ 2019-04-13 15:27:52,441][cascade_classifier.fit_transform] [layer=4] look_indexs=[0], X_cur_train.shape=(26892, 52), X_cur_test.shape=(0, 52)
[ 2019-04-13 15:27:52,718][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_4 - estimator_0 - 5_folds.train_0.predict)=99.41%
[ 2019-04-13 15:27:52,960][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_4 - estimator_0 - 5_folds.

[ 2019-04-13 15:28:07,195][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_5 - estimator_1 - 5_folds.train_2.predict)=99.42%
[ 2019-04-13 15:28:07,459][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_5 - estimator_1 - 5_folds.train_3.predict)=99.42%
[ 2019-04-13 15:28:07,703][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_5 - estimator_1 - 5_folds.train_4.predict)=99.29%
[ 2019-04-13 15:28:07,709][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_5 - estimator_1 - 5_folds.train_cv.predict)=99.36%
[ 2019-04-13 15:28:07,976][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_5 - estimator_2 - 5_folds.train_0.predict)=99.22%
[ 2019-04-13 15:28:08,238][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_5 - estimator_2 - 5_folds.train_1.predict)=99.35%
[ 2019-04-13 15:28:08,486][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_5 - estimator_2 - 5_folds.train_2.predict)=99.46%
[ 2019-04-13 15:28:08,743][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_5 - estimator_2 - 5_folds

===Accuracy===
Class 0: 99.18%
Mean±SD: 99.18±0.65%
Class 1: 99.62%
Mean±SD: 99.62±0.23%
Class 2: 99.52%
Mean±SD: 99.52±0.36%
Class 3: 99.40%
Mean±SD: 99.40±0.61%
Class 4: 98.92%
Mean±SD: 98.92±1.03%
Class 5: 99.21%
Mean±SD: 99.21±0.64%
Overall: 99.31%
Mean±SD: 99.31±0.29%
===   F1  ===
Class 0: 99.18%
Mean±SD: 99.18±0.45%
Class 1: 99.63%
Mean±SD: 99.63±0.19%
Class 2: 99.48%
Mean±SD: 99.48±0.39%
Class 3: 99.45%
Mean±SD: 99.45±0.41%
Class 4: 98.76%
Mean±SD: 98.76±0.80%
Class 5: 99.23%
Mean±SD: 99.23±0.43%
F{beta}: 99.47%
Mean±SD: 99.47±0.29%


